In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import check_array
from collections import Counter

In [2]:
def euclidean_distance(point, centroid):
    return np.sqrt(np.sum((point - centroid) ** 2))

In [3]:
# Loading an image (replace filename if you want):
image_path = 'giraffe.png'
image = cv2.imread(image_path)

# Reducing the size of the image, so that DBSCAN runs in a reasonable amount of time:
# small_image is 0.5x the size of the original. You may change this value.
image = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

height, width, _ = image.shape
pixel_data = image.reshape(-1, 3)

In [4]:
image.shape

(134, 200, 3)

In [5]:
pixel_data.shape

(26800, 3)

### Test Kmeans

In [5]:
#Kmeans parameters
k = 2
max_iterations = 100

In [ ]:
# data_points = np.random.rand(50,3) * 10
data_points = pixel_data
data_points.shape

In [7]:
test_labels = []

In [ ]:
np.random.seed(42)
centroids = data_points[np.random.choice(data_points.shape[0],k,replace=False)]
labels = []

for iteration in range(max_iterations):
    # print(f"Iteration {iteration}, Centroids = {centroids}")
    clusters = [[] for _ in range(k)]
    test_labels = []
    for point in data_points:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        cluster_index = np.argmin(distances)
        clusters[cluster_index].append(point)
        test_labels.append(cluster_index)
        # print(f'Point = {point}, distances = {distances}, cluster_index = {cluster_index}')
    # print(f'clusters = {clusters}')

    new_centroids = []
    for cluster in clusters:
        if cluster:
            new_centroid = np.mean(cluster, axis=0)
            new_centroids.append(new_centroid)
        else:
            new_centroids.append(centroids[len(new_centroids)])

    new_centroids = np.array(new_centroids)

    if np.allclose(centroids, new_centroids):
        print(f'Converged after {iteration + 1} iterations')
        break

    centroids = new_centroids
    labels = test_labels
    # print('-'*25)
    

In [ ]:
labels.count(1) + labels.count(0)

In [ ]:
for index in range(len(clusters)):
    print(f'Cluster with index {index} = {clusters[index]}, Total values = {len(clusters[index])}')

In [ ]:
clusters = [np.array(cluster) for cluster in clusters]
clusters

In [ ]:
for index, cluster in enumerate(clusters):
    print(index, cluster)

In [ ]:
labels = []
for point in data_points:
    for label, cluster in enumerate(clusters):
        if point in cluster:
            labels.append(label)

print(labels)

In [ ]:
labels.__len__()

### Test DBSCAN

In [6]:
# DBSCAN
# Setting hyperparameter(s):
eps = 5
min_pts = 30

#Load data
# data_points = np.random.randint(low=0, high=255, size=(1000,3))
data_points = pixel_data
data_points.shape

(26800, 3)

In [7]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(min_samples=min_pts,eps=eps,algorithm='brute')
dbscan_labels = dbscan.fit_predict(data_points)

In [8]:
Counter(dbscan_labels)

Counter({-1: 9139,
         0: 6062,
         2: 2991,
         13: 2610,
         3: 1913,
         8: 1758,
         10: 960,
         5: 233,
         19: 100,
         4: 97,
         24: 86,
         26: 86,
         14: 81,
         1: 74,
         20: 70,
         15: 59,
         17: 55,
         6: 55,
         21: 52,
         9: 39,
         7: 38,
         22: 37,
         16: 36,
         25: 32,
         27: 31,
         12: 30,
         11: 30,
         18: 26,
         23: 20})

In [27]:
class CustomDBSCAN:
    def __init__(self, eps=0.5, min_samples=5, metric='euclidean'):
        """
        Creates an instance of CustomDBSCAN.
        :param min_samples: Equivalent to minPts. Minimum amount of neighbors of a core object.
        :param eps: Short for epsilon. Radius of considered circle around a possible core object.
        :param metric: Used metric for measuring distances.
        """
        self.eps = eps
        self.min_samples = min_samples
        self.metric = metric
        self.labels_ = None

    def fit(self, X: np.ndarray, y=None):
        """
        This is the main clustering method of the CustomDBSCAN class, which means that this is one of the methods you
        will have to complete/implement. The method performs the clustering on vectors given in X. It is important that
        this method saves the determined labels (=mapping of vectors to clusters) in the "self.labels_" attribute! As
        long as it does this, you may change the content of this method completely and/or encapsulate the necessary
        mechanisms in additional functions.
        :param X: Array that contains the input feature vectors
        :param y: Unused
        :return: Returns the clustering object itself.
        """
        # Input validation:
        X = check_array(X, accept_sparse='csr')

        """
            Notes:
            -------
            label info

            0 - Unvisited Point
            -1 - Noise Point
            n - assigned to cluster 'n'
        """

        # Determination of labels:
        self.labels_ = None  # TODO: Implement your solution here!
        self.labels_ = [0] * X.shape[0]

        cluster_id = 0

        n = X.shape[0]

        for index in range(n):
            object = X[index]
            if self.labels_[index] == 0:
                print(f'Exploring object {object} at index {index}') 
                self._expand_cluster(X,index,cluster_id,self.eps,self.min_samples)
                cluster_id += 1
            

        return self

    def fit_predict(self, X: np.ndarray, y=None) -> np.ndarray:
        """
        Calls fit() and immediately returns the labels. See fit() for parameter information.
        """
        self.fit(X)
        return self.labels_
    
    def _get_seeds(self,X,obj_index,eps):
        seeds = []
        for index, point in enumerate(X):
            if euclidean_distance(X[obj_index],point) <= eps:
                seeds.append(index)
        return seeds
    
    def _expand_cluster(self,X,obj_index,cluster_id,eps,min_samples):
        seeds = self._get_seeds(X,obj_index,eps)
        print(seeds)

        if len(seeds) < min_samples:
            self.labels_[obj_index] = -1 # Mark as noise point
        else:
            for seed_index, seed in enumerate(seeds):
                self.labels_[seed_index] = cluster_id # Mark/Add all seeds around that core point to that cluster
            seeds.remove(obj_index)
            
            seed_index = 0
            while seed_index < len(seeds):
                new_seeds = self._get_seeds(X,seed_index,eps)
                if len(new_seeds) >= min_samples:
                    for neighbour_index in range(len(new_seeds)):
                        neighbour = new_seeds[neighbour_index]
                        if self.labels_[neighbour] in [-1,0]:
                            if self.labels_[neighbour] == 0:
                                seeds.append(neighbour)
                            self.labels_[neighbour] = cluster_id
                seeds.remove(neighbour)
                seed_index += 1
                

In [28]:
customdbscan = CustomDBSCAN(eps=eps,min_samples=2)
customdbscan_labels = customdbscan.fit_predict(X=pixel_data[:10])
# print(customdbscan_labels.shape)
print(Counter(customdbscan_labels))

Exploring object [185 162 124] at index 0
[0, 1]


IndexError: index 10 is out of bounds for axis 0 with size 10

[2]

In [8]:
def get_neighbours(point_index,data_points=data_points,radius=eps):
    neighbours = []
    for index, point in enumerate(data_points):
        if euclidean_distance(data_points[point_index], point) <= radius:
            neighbours.append(index)
    
    return neighbours

In [14]:
def expand_cluster(index, neighbours, cluster_id):
    i = 0
    while i < len(neighbours):
        neighbour = neighbours[i]
        if labels[neighbour] == -1: # already a neighbour point and marked as noise, so it'll be masked as leaf point and added to cluster
            labels[neighbour] = cluster_id
        elif labels[neighbour] == 0: # unvisited point
            labels[neighbour] = cluster_id
            new_neighbours = get_neighbours(neighbour)
            if len(new_neighbours) >= min_pts:
                neighbours  = neighbours + new_neighbours # add the new neighbours into the old neighbours
        i += 1
        print(f'{i}, Remaining Neighbous to explore : {len(neighbours)}, Clusters so far = {Counter(labels)}')
    # print(f'Clusters so far = {Counter(labels)}')

In [ ]:
# label information
# -1 = Noise Point
# 0 = Not visited yet
# n = already visited and assigned to cluster n

labels = np.full(data_points.shape[0],0)
cluster_id = 0

# Algo Start
for index, point in enumerate(data_points):
    print(f'Point = {point}, Clusters = {Counter(labels)}')
    if labels[index] == 0: 
        neighbours = get_neighbours(index)
        if len(neighbours) < min_pts:
            labels[index] = -1 # mark as noise point
        else:
            cluster_id += 1
            labels[index] = cluster_id # assign the point ot cluster
            # print(Counter(labels))
            # Grow Cluster
            # neighbours, cluster id, point
            expand_cluster(index,neighbours,cluster_id)
    # print(Counter(labels))

In [33]:
import numpy

def custom_dbscan(D, eps, MinPts):
    '''
    Cluster the dataset `D` using the DBSCAN algorithm.
    
    dbscan takes a dataset `D` (a list of vectors), a threshold distance
    `eps`, and a required number of points `MinPts`.
    
    It will return a list of cluster labels. The label -1 means noise, and then
    the clusters are numbered starting from 1.
    '''
 
    # This list will hold the final cluster assignment for each point in D.
    # There are two reserved values:
    #    -1 - Indicates a noise point
    #     0 - Means the point hasn't been considered yet.
    # Initially all labels are 0.    
    labels = [0]*len(D)

    # C is the ID of the current cluster.    
    C = 0
    
    # This outer loop is just responsible for picking new seed points--a point
    # from which to grow a new cluster.
    # Once a valid seed point is found, a new cluster is created, and the 
    # cluster growth is all handled by the 'expandCluster' routine.
    
    # For each point P in the Dataset D...
    # ('P' is the index of the datapoint, rather than the datapoint itself.)
    for P in range(0, len(D)):
    
        # Only points that have not already been claimed can be picked as new 
        # seed points.    
        # If the point's label is not 0, continue to the next point.
        if not (labels[P] == 0):
           continue
        
        # Find all of P's neighboring points.
        NeighborPts = region_query(D, P, eps)
        
        # If the number is below MinPts, this point is noise. 
        # This is the only condition under which a point is labeled 
        # NOISE--when it's not a valid seed point. A NOISE point may later 
        # be picked up by another cluster as a boundary point (this is the only
        # condition under which a cluster label can change--from NOISE to 
        # something else).
        if len(NeighborPts) < MinPts:
            labels[P] = -1
        # Otherwise, if there are at least MinPts nearby, use this point as the 
        # seed for a new cluster.    
        else: 
           C += 1
           grow_cluster(D, labels, P, NeighborPts, C, eps, MinPts)
        print('-'*50)
    
    # All data has been clustered!
    return labels


def grow_cluster(D, labels, P, NeighborPts, C, eps, MinPts):
    '''
    Grow a new cluster with label `C` from the seed point `P`.
    
    This function searches through the dataset to find all points that belong
    to this new cluster. When this function returns, cluster `C` is complete.
    
    Parameters:
      `D`      - The dataset (a list of vectors)
      `labels` - List storing the cluster labels for all dataset points
      `P`      - Index of the seed point for this new cluster
      `NeighborPts` - All of the neighbors of `P`
      `C`      - The label for this new cluster.  
      `eps`    - Threshold distance
      `MinPts` - Minimum required number of neighbors
    '''

    # Assign the cluster label to the seed point.
    labels[P] = C
    
    # Look at each neighbor of P (neighbors are referred to as Pn). 
    # NeighborPts will be used as a FIFO queue of points to search--that is, it
    # will grow as we discover new branch points for the cluster. The FIFO
    # behavior is accomplished by using a while-loop rather than a for-loop.
    # In NeighborPts, the points are represented by their index in the original
    # dataset.
    i = 0
    while i < len(NeighborPts):    
        
        # Get the next point from the queue.        
        Pn = NeighborPts[i]
       
        # If Pn was labelled NOISE during the seed search, then we
        # know it's not a branch point (it doesn't have enough neighbors), so
        # make it a leaf point of cluster C and move on.
        if labels[Pn] == -1:
           labels[Pn] = C
        
        # Otherwise, if Pn isn't already claimed, claim it as part of C.
        elif labels[Pn] == 0:
            # Add Pn to cluster C (Assign cluster label C).
            labels[Pn] = C
            
            # Find all the neighbors of Pn
            PnNeighborPts = region_query(D, Pn, eps)
            
            # If Pn has at least MinPts neighbors, it's a branch point!
            # Add all of its neighbors to the FIFO queue to be searched. 
            if len(PnNeighborPts) >= MinPts:
                NeighborPts = NeighborPts + PnNeighborPts
            # If Pn *doesn't* have enough neighbors, then it's a leaf point.
            # Don't queue up it's neighbors as expansion points.
            #else:
                # Do nothing                
                #NeighborPts = NeighborPts               
        
        # Advance to the next point in the FIFO queue.
        print(f'{i}, Remaining Neighbous to explore : {len(NeighborPts)}, Clusters so far = {Counter(labels)}')
        i += 1

    
    # We've finished growing cluster C!


def region_query(D, P, eps):
    '''
    Find all points in dataset `D` within distance `eps` of point `P`.
    
    This function calculates the distance between a point P and every other 
    point in the dataset, and then returns only those points which are within a
    threshold distance `eps`.
    '''
    neighbors = []
    
    # For each point in the dataset...
    for Pn in range(0, len(D)):
        
        # If the distance is below the threshold, add it to the neighbors list.
        if numpy.linalg.norm(D[P] - D[Pn]) < eps:
           neighbors.append(Pn)
            
    return neighbors

In [ ]:
labels = custom_dbscan(D=data_points,eps=eps,MinPts=min_pts)